In [ ]:
from pathlib import Path

ROOT = Path('../..').resolve()

In [ ]:
import geopandas as gpd

In [ ]:
las = gpd.GeoDataFrame.from_file(ROOT / "data/raw/Local_Authority_Districts_May_2024_Boundaries_UK_BGC_-7565825659957132419.geojson")

In [ ]:
bmdc = las[las.LAD24CD == "E08000032"].to_crs(crs=4326)

In [ ]:
bmdc

In [ ]:
bmdc.to_file(ROOT / "data/raw/bd_lad.geojson")

In [ ]:
import petl as etl

In [ ]:
DATA_PATH = ROOT / 'data/raw'

In [ ]:
bd25 = etl.fromcsv(
    DATA_PATH / "bd25_geo.csv"
).split('PCDS', r'\s+', ['OUT', 'IN'])


In [ ]:
bd25

In [ ]:
def reducer(k, rows):
  all = list(rows)
  return (
    k,
    len([r for r in all if r.OSLAUA == "E08000032"]),
    len([r for r in all if r.OSLAUA != "E08000032"])
  )

counts = bd25.rowreduce(
  'OUT', reducer,
  header=["postcode", "inside", "outside"]
)

In [ ]:
counts.tocsv(ROOT / 'data/raw/postcode_stats.csv')

In [ ]:
counts

In [ ]:
# https://geoportal.statistics.gov.uk/

In [ ]:
wards = gpd.GeoDataFrame.from_file(ROOT / "data/raw/Wards_May_2024_Boundaries_UK_BSC_-6022693383694477628.geojson").to_crs(crs=4326)

In [ ]:
wards

In [ ]:
import pandas as pd
ward_lkp = pd.read_csv(ROOT / 'data/raw/wards.csv')

In [ ]:
ward_lkp.Code

In [ ]:
wards[wards.WD24CD.isin(ward_lkp.Code)].to_file(ROOT / 'data/raw/wards.geojson')

In [ ]:
msoa = gpd.GeoDataFrame.from_file(ROOT / 'data/raw/MSOA_DEC_2021_EW_NC_v3_2768211653661228621.geojson').to_crs(crs=4326)

In [ ]:
msoa[msoa.MSOA21NM.str.startswith('Bradford')]